In [15]:
!pip install pandas requests

In [16]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
import json

In [17]:
twitter_data = []
key = os.getenv("SCRAPER_API_KEY")
payload = {
    'api_key': key,
    'query': 'mpox',
    'num': '25',
}
response = requests.get(
    'https://api.scraperapi.com/structured/twitter/search', params=payload
)

data = response.json()

In [18]:
data

{'search_information': {'query_displayed': 'site:twitter.com "mpox"'},
 'organic_results': [{'position': 0,
   'title': 'Hans Kluge',
   'snippet': 'As we tackle #mpox as a public health emergency of international concern for the second time in 2 years, I want to convey 3 basic messages\xa0...',
   'highlighs': ['mpox'],
   'link': 'https://twitter.com/hans_kluge/status/1825905477735120903',
   'displayed_link': 'https://twitter.com › hans_kluge › status'},
  {'position': 1,
   'title': 'World Health Organization (WHO) - X.com',
   'snippet': 'Learn about #mpox and how WHO is working with partners to contain the current mpox outbreak in #DRC.',
   'highlighs': ['mpox', 'mpox'],
   'link': 'https://twitter.com/WHO/status/1824705282955485335',
   'displayed_link': '890+ likes  ·  1 month ago'},
  {'position': 2,
   'title': 'World Health Organization (WHO)',
   'snippet': 'What is #mpox? Learn more  http://bit.ly/3AvAOwf. Mpox (previously monkeypox) is a viral infection which can spread 

In [19]:
data.keys()

dict_keys(['search_information', 'organic_results', 'images', 'pagination'])

In [20]:
data['organic_results']

[{'position': 0,
  'title': 'Hans Kluge',
  'snippet': 'As we tackle #mpox as a public health emergency of international concern for the second time in 2 years, I want to convey 3 basic messages\xa0...',
  'highlighs': ['mpox'],
  'link': 'https://twitter.com/hans_kluge/status/1825905477735120903',
  'displayed_link': 'https://twitter.com › hans_kluge › status'},
 {'position': 1,
  'title': 'World Health Organization (WHO) - X.com',
  'snippet': 'Learn about #mpox and how WHO is working with partners to contain the current mpox outbreak in #DRC.',
  'highlighs': ['mpox', 'mpox'],
  'link': 'https://twitter.com/WHO/status/1824705282955485335',
  'displayed_link': '890+ likes  ·  1 month ago'},
 {'position': 2,
  'title': 'World Health Organization (WHO)',
  'snippet': 'What is #mpox? Learn more  http://bit.ly/3AvAOwf. Mpox (previously monkeypox) is a viral infection which can spread between people, mainly.',
  'highlighs': ['mpox', 'Mpox'],
  'link': 'https://twitter.com/WHO/status/1825

In [21]:
allTweets = data['organic_results']
for tweet in allTweets:
    twitter_data.append(tweet)

In [22]:
df = pd.DataFrame(twitter_data)
df.to_json("tweets.json", orient='index')
df

,position,title,snippet,highlighs,link,displayed_link
0,0,Hans Kluge,As we tackle #mpox as a public health emergenc...,[mpox],https://twitter.com/hans_kluge/status/18259054...,https://twitter.com › hans_kluge › status
1,1,World Health Organization (WHO) - X.com,Learn about #mpox and how WHO is working with ...,"[mpox, mpox]",https://twitter.com/WHO/status/182470528295548...,890+ likes · 1 month ago
2,2,World Health Organization (WHO),What is #mpox? Learn more http://bit.ly/3AvAO...,"[mpox, Mpox]",https://twitter.com/WHO/status/182557073510178...,1.1K+ likes · 1 month ago
3,3,Hans Kluge,Sweden – a @WHO_Europe Member State – has conf...,[mpox],https://twitter.com/hans_kluge/status/18241205...,260+ likes · 1 month ago
4,4,World Health Organization (WHO) - X.com,#Mpox transmission occurs via close contact wi...,[Mpox],https://twitter.com/WHO/status/182622586689801...,570+ likes · 4 weeks ago
5,5,DC Health,"Anyone can catch mpox, but if you or someone y...","[mpox, mpox]",https://twitter.com/_DCHealth/status/183603263...,1 day ago
6,6,United Nations,Mpox - formerly monkeypox - is a rare but dang...,"[Mpox, mpox]",https://twitter.com/UN/status/1823843541094678780,180+ likes · 1 month ago
7,7,njdoh - X.com,Mpox can spread through direct contact with ra...,"[Mpox, mpox]",https://twitter.com/NJDeptofHealth/status/1836...,8 hours ago
8,8,Ministry of Health,Mpox is a viral illness that can be managed wi...,[Mpox],https://twitter.com/MoHFW_INDIA/status/1833578...,10+ likes · 1 week ago
9,9,WHO African Region - WHOAFRO,"What are the symptoms of #mpox, and how is it ...",[mpox],https://twitter.com/WHOAFRO/status/18363142157...,6 likes · 17 hours ago


In [24]:
pip install ntscraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 9.2 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [25]:
from ntscraper import Nitter

In [26]:
scraper = Nitter()

Testing instances: 100%|██████████| 16/16 [00:19<00:00,  1.21s/it]


In [29]:
def getTweets(name, modes, num):
    tweets = scraper.get_tweets(name, mode=modes, number=num)

    print(tweets)
    final_tweets = []
    for tweet in tweets['tweets']:
        data = [tweet['link'], tweet['text'], tweet['date']]
        final_tweets.append(data)
    data = pd.DataFrame(final_tweets, columns=['link', 'text', 'date'])
    return data

In [30]:
data = getTweets('mpox', 'term', 25)

18-Sep-24 20:54:29 - No instance specified, using random instance https://nitter.lucabased.xyz
18-Sep-24 20:54:50 - Current stats for mpox: 15 tweets, 0 threads...
18-Sep-24 20:55:01 - Current stats for mpox: 24 tweets, 0 threads...
18-Sep-24 20:55:06 - Current stats for mpox: 25 tweets, 0 threads...
{'tweets': [{'link': 'https://twitter.com/CovidMalinfo/status/1836584531966505165#m', 'text': 'I feel like this is how they plan to spread mpox so they can in turn justify mass vaccination (again) through fear mongering.', 'user': {'name': 'CovidMalinformation', 'username': '@CovidMalinfo', 'profile_id': '1788278173127315456', 'avatar': 'https://pbs.twimg.com/profile_images/1788278173127315456/olXhdj2J_bigger.jpg'}, 'date': 'Sep 19, 2024 · 1:54 AM UTC', 'is-retweet': False, 'is-pinned': False, 'external-link': '', 'replying-to': ['@BretWeinstein'], 'quoted-post': {}, 'stats': {'comments': 0, 'retweets': 0, 'quotes': 0, 'likes': 0}, 'pictures': [], 'videos': [], 'gifs': []}, {'link': 'https

In [31]:
data

,link,text,date
0,https://twitter.com/CovidMalinfo/status/183658...,I feel like this is how they plan to spread mp...,"Sep 19, 2024 · 1:54 AM UTC"
1,https://twitter.com/pulso_periodico/status/183...,¿Sabías que la OMS ha declarado una emergencia...,"Sep 18, 2024 · 1:53 PM UTC"
2,https://twitter.com/RYVCP_Kamembe/status/18364...,None kimwe n'ibindi bihe muri iyi minsi #RYVCP...,"Sep 18, 2024 · 6:18 PM UTC"
3,https://twitter.com/HausofHilton/status/183605...,"Beza mpox dengan chicken pox Untuk mpox, rua...","Sep 17, 2024 · 2:45 PM UTC"
4,https://twitter.com/VigilantNews/status/183647...,"FDA Approves Mpox Vaccine, Warns It May Cause ...","Sep 18, 2024 · 6:45 PM UTC"
5,https://twitter.com/engelvonleber/status/18365...,Ok but only a certain subgroup of people are s...,"Sep 19, 2024 · 12:50 AM UTC"
6,https://twitter.com/JonnyGotThatDoh/status/183...,Mpox isthe swamps round 2 attempt to lock us d...,"Sep 18, 2024 · 9:57 PM UTC"
7,https://twitter.com/newstart_2024/status/18362...,"🤔Mpox vaccine is 58% effective after 1 dose, s...","Sep 18, 2024 · 5:36 AM UTC"
8,https://twitter.com/SltcPonk3/status/183658339...,et qd tu parles a des meufs de 15 ans c ps de ...,"Sep 19, 2024 · 1:49 AM UTC"
9,https://twitter.com/eannaso/status/18357486244...,"According to the @UN ""The @WHO (WHO) has appro...","Sep 16, 2024 · 6:32 PM UTC"


In [32]:
from bs4 import BeautifulSoup

In [35]:
def scrape_full_tweet(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    tweet_text = soup.find('meta', {'property': 'og:description'})['content']
    return tweet_text

In [34]:
def getTweets(name, modes, num):
    
    payload = {
        'api_key': os.getenv("SCRAPER_API_KEY"),
        'query': name,
        'num': num
    }
    response = requests.get(
        'https://api.scraperapi.com/structured/twitter/search', params=payload
    )

    total = response.json()
    final_tweets = []
    if 'organic_results' in total:
        for tweet in total['organic_results']:
            link = tweet['link']
            date = tweet.get('date', 'Unknown')
            full_text = scrape_full_tweet(link)
            final_tweets.append([link, full_text, date])

        data = pd.DataFrame(final_tweets, columns=['link', 'text', 'date'])
        return data
    else:
        print("No tweets found or API response doesn't contain expected data.")
        return None

In [37]:
data = getTweets('mpox', 'term', 25)
print(data)

18-Sep-24 21:08:54 - No instance specified, using random instance https://nitter.lucabased.xyz
18-Sep-24 21:09:01 - Current stats for mpox: 16 tweets, 0 threads...
18-Sep-24 21:09:06 - Current stats for mpox: 25 tweets, 0 threads...
{'tweets': [{'link': 'https://twitter.com/VigilantFox/status/1836525559758950556#m', 'text': 'Read More:  FDA Approves Vaccine for Mpox — Warns It May Cause Death in Vaccinated  https://childrenshealthdefense.org/defender/fda-acam2000-mpox-vaccine-warning-death-unvaccinated/?utm_source=luminate&utm_medium=email&utm_campaign=defender&utm_id=20240917', 'user': {'name': 'The Vigilant Fox 🦊', 'username': '@VigilantFox', 'profile_id': '1657046545265557504', 'avatar': 'https://pbs.twimg.com/profile_images/1657046545265557504/7wQ3wrAv_bigger.jpg'}, 'date': 'Sep 18, 2024 · 9:59 PM UTC', 'is-retweet': True, 'is-pinned': False, 'external-link': '', 'replying-to': [], 'quoted-post': {}, 'stats': {'comments': 4, 'retweets': 54, 'quotes': 8, 'likes': 84}, 'pictures': []

In [38]:
pip install tweepy


Note: you may need to restart the kernel to use updated packages.


In [39]:
import tweepy